# Registry Persistence via Explorer Run Key
Detects a possible persistence mechanism using RUN key for Windows Explorer and poiting to a suspicious folder

## Rule Content
```
- title: Registry Persistence via Explorer Run Key
  id: b7916c2a-fa2f-4795-9477-32b731f70f11
  status: experimental
  description: Detects a possible persistence mechanism using RUN key for Windows
    Explorer and poiting to a suspicious folder
  author: Florian Roth
  date: 2018/07/18
  references:
  - https://researchcenter.paloaltonetworks.com/2018/07/unit42-upatre-continues-evolve-new-anti-analysis-techniques/
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 13
      TargetObject: '*\Microsoft\Windows\CurrentVersion\Policies\Explorer\Run'
      Details:
      - C:\Windows\Temp\\*
      - C:\ProgramData\\*
      - '*\AppData\\*'
      - C:\$Recycle.bin\\*
      - C:\Temp\\*
      - C:\Users\Public\\*
      - C:\Users\Default\\*
    condition: selection
  tags:
  - attack.persistence
  - attack.t1060
  - capec.270
  fields:
  - Image
  - ParentImage
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"13" AND registry_key_path.keyword:*\\Microsoft\\Windows\\CurrentVersion\\Policies\\Explorer\\Run AND registry_key_value.keyword:(C\:\\Windows\\Temp\\* OR C\:\\ProgramData\\* OR *\\AppData\\* OR C\:\\$Recycle.bin\\* OR C\:\\Temp\\* OR C\:\\Users\\Public\\* OR C\:\\Users\\Default\\*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()